In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
import logging
tf.get_logger().setLevel(logging.ERROR)
import time

In [3]:
splits = tfds.Split.ALL.subsplit(weighted=(80,20))
(train_dataset,validation_dataset),info = tfds.load('cats_vs_dogs',split=splits,with_info=True,
                                                    as_supervised=True)

In [6]:
def format_image(image,label):
    image = tf.image.resize(image,(IMG_SIZE,IMG_SIZE))
    return image,label
IMG_SIZE = 224
BATCH_SIZE = 32
num_examples = info.splits['train'].num_examples
train_batches = train_dataset.cache().shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_dataset.cache().map(format_image).batch(BATCH_SIZE).prefetch(1)

In [7]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,input_shape=(IMG_SIZE,IMG_SIZE,3),trainable=False)

In [9]:
model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(2,activation='softmax')
]).compile(loss = 'sparse_categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])

epochs = 3
history = model.fit(train_batches,
                   validation_data=validation_batches,
                   epochs=epochs)

In [ ]:
class_names = numpy.array(info.features['label'].names)
image_batch,label_batch = next(iter(train_batches.take(1)))
image_batch = image_batch.numpy()
label_batch = label_batch.numpy()
predicted_batch = tf.squeeze(model.predict(image_batch)).numpy()
predicted_ids = np.argmax(predicted_batch,axis=-1)
predicted_class_names = class_names[predicted_idsb]

In [ ]:
t = time.time()
export_path_keras = "./{}.h5".format(int(t))
model.save(export_path_keras)

In [ ]:
reloaded = tf.keras.models.load_model(export_path_keras,
                                     custom_objects={'KerasLayer':hub.KerasLayer})
result_batch = model.predict(image_batch)
reloaded_batch = reloaded.predict(image_batch)

In [ ]:
history = reloaded.fi(train_batches,
                      validation_data=validation_batches,
                      epochs=epochs)
